## Train model
Danh sách class: <br>
    0. Correct <br>
    1. Chân quá hẹp <br >
    2. Chân quá rộng <br >
    3. Khoảng cách giữa 2 đầu gối quá nhỏ <br >
    4. Xuống quá xâu <br >
    5. Lỗi gập gập lưng <br >

In [40]:
import mediapipe as mp
import cv2
import pandas as pd
import pickle
import numpy as np
import csv
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 1. Khởi tạo keypoints và các phương thức 
Các keypoints cần lấy :
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",

Dataframe gồm label và các features là toạ độ keypoints gồm 4 thành phần: x, y, z, visibility
- x, y: tọa độ x, y của keypoints đã được chuẩn hóa min max scale
- z: độ sâu của điểm ảnh theo điểm giữa hông 
- visibility: biểu thị khả năng có thể nhìn thấy hoặc che khuất
 

In [41]:
#Chọn các điểm cần lấy
Important_kp = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
]

#Tạo header cho dataframe
header = ["label"]

for kp in Important_kp:
    header.extend([f"{kp}_x", f"{kp}_y", f"{kp}_z", f"{kp}_visibility"])

header

['label',
 'NOSE_x',
 'NOSE_y',
 'NOSE_z',
 'NOSE_visibility',
 'LEFT_SHOULDER_x',
 'LEFT_SHOULDER_y',
 'LEFT_SHOULDER_z',
 'LEFT_SHOULDER_visibility',
 'RIGHT_SHOULDER_x',
 'RIGHT_SHOULDER_y',
 'RIGHT_SHOULDER_z',
 'RIGHT_SHOULDER_visibility',
 'LEFT_HIP_x',
 'LEFT_HIP_y',
 'LEFT_HIP_z',
 'LEFT_HIP_visibility',
 'RIGHT_HIP_x',
 'RIGHT_HIP_y',
 'RIGHT_HIP_z',
 'RIGHT_HIP_visibility',
 'LEFT_KNEE_x',
 'LEFT_KNEE_y',
 'LEFT_KNEE_z',
 'LEFT_KNEE_visibility',
 'RIGHT_KNEE_x',
 'RIGHT_KNEE_y',
 'RIGHT_KNEE_z',
 'RIGHT_KNEE_visibility',
 'LEFT_ANKLE_x',
 'LEFT_ANKLE_y',
 'LEFT_ANKLE_z',
 'LEFT_ANKLE_visibility',
 'RIGHT_ANKLE_x',
 'RIGHT_ANKLE_y',
 'RIGHT_ANKLE_z',
 'RIGHT_ANKLE_visibility']

In [42]:
from math import sqrt

# rescale frame 1/2
def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

#Tính khoảng cách giữa 2 điểm trong không gian 2D
def calculate_distance(pointX, pointY) -> float:
    '''
    Calculate a distance between 2 points
    '''

    x1, y1 = pointX
    x2, y2 = pointY

    return sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

#Tính góc giữa 3 điểm trong không gian 3D
def calculate_angle(pointA, pointB, pointC) -> float:
    """
    Tính góc giữa ba điểm trong không gian 3D (đơn vị: độ)
    a, b, c là danh sách chứa tọa độ 3D: [x, y, z]
    """

    #Độ sâu ảnh
    t = 33;
    A = np.array([pointA[0], pointA[1], pointA[2]/2])  # (xA, yA, zA)
    B = np.array([pointB[0], pointB[1], pointB[2]/2 ])  # (xB, yB, zB)
    C = np.array([pointC[0], pointC[1], pointC[2]/2])  # (xC, yC, zC)

    # Vector BA và BC
    BA = A - B
    BC = C - B

    # Tính tích vô hướng
    dot_product = np.dot(BA, BC)
    
    # Tính độ dài vector
    norm_BA = np.linalg.norm(BA)
    norm_BC = np.linalg.norm(BC)

    # Tính góc bằng công thức cos(theta) = (A.B) / (|A| * |B|)
    cos_theta = dot_product / (norm_BA * norm_BC)
    
    # Chuyển từ radian sang độ
    angle = np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

    return angle

#Đọc dữ liệu từ frame trả về points
def extract_important_keypoints(results) -> list:
    '''
    Extract important keypoints from mediapipe pose detection
    '''
    landmarks = results.pose_landmarks.landmark

    data = []
    for lm in Important_kp:
        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
        data.append([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
    
    return np.array(data).flatten().tolist()


In [43]:
#Lỗi 1 - 2: ratio = feet/shoulder
Min_radio_feet_shoulder = 0.59
Max_radio_feet_shoulder = 1

#Lỗi 3 : ratio = knee/feet
Min_up_radio_knee_feet = 0.88
Min_middle_ratio_knee_feet = 0.96 
Min_down_ratio_knee_feet = 1.42

#Lỗi 4: knee angle < 50
Min_knee_angle = 50

#Lỗi 5: ratio = knee_angle/hip_angle
Max_ratio_knee_hip = 0.98


In [44]:
df = pd.DataFrame(columns=header)
df_test = pd.DataFrame(columns=header)
df_va = pd.DataFrame(columns=header)
df 

,label,NOSE_x,NOSE_y,NOSE_z,NOSE_visibility,LEFT_SHOULDER_x,LEFT_SHOULDER_y,LEFT_SHOULDER_z,LEFT_SHOULDER_visibility,RIGHT_SHOULDER_x,...,RIGHT_KNEE_z,RIGHT_KNEE_visibility,LEFT_ANKLE_x,LEFT_ANKLE_y,LEFT_ANKLE_z,LEFT_ANKLE_visibility,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_ANKLE_z,RIGHT_ANKLE_visibility


# 2. Hàm duyệt data lỗi chân quá hẹp

In [45]:
#Xử lý từng video để tính khoảng cách chân, vai
def detection_1_error(Video_folder, Video_name, type):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    images = []  # Danh sách lưu hình ảnh
    max_images = 10  # Số hình ảnh tối đa hiển thị cùng lúc

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích xuất kpkp
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            #khởi tạo biến tính khoảng cách
            shoulder_width = feet_width = None
            
            try:
                # Lấy kp từ frame
                # row = extract_important_keypoints(results)
                # X = pd.DataFrame([row], columns=header[1:])

                # Tinhs toán và so sánh khoảng cách vai, hông, chân
                landmarks = results.pose_landmarks.landmark
                # Khoảng cách giữa 2 vai
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

                shoulder_width = calculate_distance(left_shoulder, right_shoulder)

                # Khoảng cách giữa 2 chân (sửa từ foot_index thành ankle)
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

                feet_width = calculate_distance(left_ankle, right_ankle)


                # Kiểm tra lỗi
                row = []
                if(feet_width/shoulder_width < Min_radio_feet_shoulder):
                    row.append(1)
                    for lm in Important_kp:
                        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
                        row.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])

                if len(row) == len(df.columns):
                    if type == "train":
                        df.loc[len(df)] = row
                    elif type == "test":
                        df_test.loc[len(df_test)] = row
                    elif type == "validate":
                        df_va.loc[len(df_va)] = row
                else:
                    print(f"Lỗi! row có {len(row)} phần tử nhưng df có {len(df.columns)} cột.")

                # Nền
                cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

                # Display feet distance
                cv2.putText(image, "FEET", (15, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(feet_width, 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display shoulder distance
                cv2.putText(image, "SHOULDER", (95, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(shoulder_width, 2)), (90, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display label
                cv2.putText(image, "LABEL", (200, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(row[0], 2)), (200, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            images.append(image)

            # Khi đủ 10 ảnh, hiển thị tất cả cùng lúc
            # if len(images) == max_images:
            #     fig, axes = plt.subplots(5, 2, figsize=(32, 18))  # Tạo grid 2x5
            #     for i, ax in enumerate(axes.flat):
            #         ax.imshow(images[i])
            #         ax.set_aspect('auto')
            #         ax.axis("off")  # Ẩn trục tọa độ
            #     plt.show()
            #     images = []  # Reset danh sách sau khi hiển thị

        Cap.release()

In [46]:
import os

video_folder = "Data/Train/Chan_qua_hep"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_1_error(video_folder, video_file, "train")

video_folder = "Data/Test/Chan_qua_hep"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_1_error(video_folder, video_file, "test")

video_folder = "Data/Validation/Chan_qua_hep"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_1_error(video_folder, video_file, "validate")


Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: li

In [47]:
df 

,label,NOSE_x,NOSE_y,NOSE_z,NOSE_visibility,LEFT_SHOULDER_x,LEFT_SHOULDER_y,LEFT_SHOULDER_z,LEFT_SHOULDER_visibility,RIGHT_SHOULDER_x,...,RIGHT_KNEE_z,RIGHT_KNEE_visibility,LEFT_ANKLE_x,LEFT_ANKLE_y,LEFT_ANKLE_z,LEFT_ANKLE_visibility,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_ANKLE_z,RIGHT_ANKLE_visibility
0,1.0,0.535062,0.249332,-0.298349,0.999890,0.567664,0.323744,-0.071113,0.999684,0.477610,...,-0.019138,0.992065,0.558640,0.903493,0.080078,0.970077,0.513414,0.908219,0.124308,0.983346
1,1.0,0.535019,0.256015,-0.305654,0.999889,0.567926,0.326266,-0.081536,0.999688,0.478063,...,-0.040403,0.992278,0.558611,0.903518,0.091800,0.970177,0.512646,0.909850,0.102087,0.983742
2,1.0,0.535222,0.263654,-0.307573,0.999891,0.567934,0.334535,-0.082262,0.999697,0.478092,...,-0.034905,0.992214,0.558633,0.901283,0.092424,0.970170,0.512343,0.909795,0.107190,0.983548
3,1.0,0.535212,0.279500,-0.308483,0.999881,0.567779,0.343994,-0.084086,0.999694,0.478039,...,-0.032856,0.991776,0.558793,0.898668,0.109657,0.968650,0.512348,0.909864,0.111147,0.982737
4,1.0,0.535222,0.287921,-0.310271,0.999887,0.567733,0.356836,-0.086673,0.999710,0.477941,...,-0.026602,0.991716,0.558939,0.897077,0.102962,0.969260,0.512467,0.910724,0.133688,0.982723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1934,1.0,0.480709,0.151462,-0.304861,0.999981,0.526945,0.260478,-0.084773,0.999830,0.428703,...,0.031203,0.996730,0.491099,0.890934,0.208882,0.990742,0.455111,0.893892,0.205204,0.995438
1935,1.0,0.480790,0.151496,-0.304818,0.999981,0.526946,0.260702,-0.084558,0.999829,0.428731,...,0.031312,0.996726,0.491095,0.891209,0.208973,0.990803,0.455155,0.894002,0.205221,0.995427
1936,1.0,0.480804,0.151605,-0.300575,0.999981,0.526949,0.260844,-0.081109,0.999831,0.428785,...,0.031301,0.996716,0.491092,0.891493,0.208588,0.990877,0.455188,0.894193,0.204733,0.995404
1937,1.0,0.480813,0.151777,-0.295606,0.999981,0.526992,0.260947,-0.077437,0.999832,0.428822,...,0.031008,0.996712,0.491091,0.891643,0.207617,0.990930,0.455219,0.894289,0.202291,0.995391


# 2. Lỗi chân rộng


In [48]:
#Xử lý từng video để tính khoảng cách chân, vai
def detection_2_error(Video_folder, Video_name, type):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    images = []  # Danh sách lưu hình ảnh
    max_images = 10  # Số hình ảnh tối đa hiển thị cùng lúc

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích xuất kpkp
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            #khởi tạo biến tính khoảng cách
            shoulder_width = feet_width = None
            
            try:
                # Lấy kp từ frame
                # row = extract_important_keypoints(results)
                # X = pd.DataFrame([row], columns=header[1:])

                # Tinhs toán và so sánh khoảng cách vai, hông, chân
                landmarks = results.pose_landmarks.landmark
                # Khoảng cách giữa 2 vai
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

                shoulder_width = calculate_distance(left_shoulder, right_shoulder)

                # Khoảng cách giữa 2 chân
                left_foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
                right_foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]

                feet_width = calculate_distance(left_foot_index, right_foot_index)

                # Kiểm tra lỗi
                row = []
                if(feet_width/shoulder_width > Max_radio_feet_shoulder):
                    row.append(2)
                    for lm in Important_kp:
                        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
                        row.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])

                if len(row) == len(df.columns):
                    if type == "train":
                        df.loc[len(df)] = row
                    elif type == "test":
                        df_test.loc[len(df_test)] = row
                    elif type == "validate":
                        df_va.loc[len(df_va)] = row
                else:
                    print(f"Lỗi! row có {len(row)} phần tử nhưng df có {len(df.columns)} cột.")

                # Nền
                cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

                # Display feet distance
                cv2.putText(image, "FEET", (15, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(feet_width, 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display shoulder distance
                cv2.putText(image, "SHOULDER", (95, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(shoulder_width, 2)), (90, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display label
                cv2.putText(image, "LABEL", (200, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(row[0], 2)), (200, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            images.append(image)

            # Khi đủ 10 ảnh, hiển thị tất cả cùng lúc
            # if len(images) == max_images:
            #     fig, axes = plt.subplots(5, 2, figsize=(32, 18))  # Tạo grid 2x5
            #     for i, ax in enumerate(axes.flat):
            #         ax.imshow(images[i])
            #         ax.set_aspect('auto')
            #         ax.axis("off")  # Ẩn trục tọa độ
            #     plt.show()
            #     images = []  # Reset danh sách sau khi hiển thị

        Cap.release()

In [49]:
import os

video_folder = "Data/Train/Chan_qua_rong"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_2_error(video_folder, video_file, "train")

video_folder = "Data/Test/Chan_qua_rong"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_2_error(video_folder, video_file, "test")

video_folder = "Data/Validation/Chan_qua_rong"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_2_error(video_folder, video_file, "validate")

In [50]:
df

,label,NOSE_x,NOSE_y,NOSE_z,NOSE_visibility,LEFT_SHOULDER_x,LEFT_SHOULDER_y,LEFT_SHOULDER_z,LEFT_SHOULDER_visibility,RIGHT_SHOULDER_x,...,RIGHT_KNEE_z,RIGHT_KNEE_visibility,LEFT_ANKLE_x,LEFT_ANKLE_y,LEFT_ANKLE_z,LEFT_ANKLE_visibility,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_ANKLE_z,RIGHT_ANKLE_visibility
0,1.0,0.535062,0.249332,-0.298349,0.999890,0.567664,0.323744,-0.071113,0.999684,0.477610,...,-0.019138,0.992065,0.558640,0.903493,0.080078,0.970077,0.513414,0.908219,0.124308,0.983346
1,1.0,0.535019,0.256015,-0.305654,0.999889,0.567926,0.326266,-0.081536,0.999688,0.478063,...,-0.040403,0.992278,0.558611,0.903518,0.091800,0.970177,0.512646,0.909850,0.102087,0.983742
2,1.0,0.535222,0.263654,-0.307573,0.999891,0.567934,0.334535,-0.082262,0.999697,0.478092,...,-0.034905,0.992214,0.558633,0.901283,0.092424,0.970170,0.512343,0.909795,0.107190,0.983548
3,1.0,0.535212,0.279500,-0.308483,0.999881,0.567779,0.343994,-0.084086,0.999694,0.478039,...,-0.032856,0.991776,0.558793,0.898668,0.109657,0.968650,0.512348,0.909864,0.111147,0.982737
4,1.0,0.535222,0.287921,-0.310271,0.999887,0.567733,0.356836,-0.086673,0.999710,0.477941,...,-0.026602,0.991716,0.558939,0.897077,0.102962,0.969260,0.512467,0.910724,0.133688,0.982723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4090,2.0,0.512599,0.170784,-0.256588,0.999996,0.565641,0.282332,-0.039706,0.999980,0.463723,...,-0.012392,0.998173,0.611001,0.914221,0.182832,0.996932,0.415025,0.918326,0.195832,0.997662
4091,2.0,0.512604,0.170828,-0.258020,0.999996,0.565666,0.282348,-0.040138,0.999979,0.463728,...,-0.010829,0.998174,0.611088,0.914725,0.182786,0.996924,0.415027,0.918701,0.195895,0.997670
4092,2.0,0.512658,0.170774,-0.257240,0.999996,0.565671,0.282355,-0.039940,0.999979,0.463732,...,-0.010219,0.998168,0.611137,0.914791,0.182662,0.996904,0.415029,0.918997,0.195525,0.997665
4093,2.0,0.512583,0.170419,-0.255323,0.999996,0.565675,0.282372,-0.039689,0.999979,0.463743,...,-0.010150,0.998158,0.611143,0.914750,0.182363,0.996883,0.415024,0.919006,0.195029,0.997654


# 3. Lỗi gối hẹp


In [51]:
#import model để dự đoán giai đoạn
import pickle


with open("LR_Up_Down_model.pkl", "rb") as f:
    trained_model = pickle.load(f)

In [52]:
#Xử lý từng video để tính khoảng cách chân, vai
def detection_3_error(Video_folder = "Data/train/Dau_goi_hep" , Video_name = "20250228_092009000_iOS.mp4" , type = "train"):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    images = []  # Danh sách lưu hình ảnh
    max_images = 10  # Số hình ảnh tối đa hiển thị cùng lúc

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích xuất kpkp
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            # Chuyển lại ảnh sang BGR để hiển thị
            # image.flags.writeable = True
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            #khởi tạo biến tính khoảng cách
            knee_width = feet_width = None
            
            try:
                # Lấy kp từ frame
                row = extract_important_keypoints(results)
                X = pd.DataFrame([row], columns=header[1:])

                # Tinhs toán và so sánh khoảng cách đầu gối, chân
                landmarks = results.pose_landmarks.landmark

                # Lấy trạng thái hiện tại 
                predicted_class = trained_model.predict(X)[0]
                prediction_probability = trained_model.predict_proba(X)[0]

                current_stage ="middle"
                if prediction_probability[prediction_probability.argmax()] >= 0.7: # Nếu xác suất dự đoán lớn hơn 0.5 thì mới cập nhật trạng thái
                    if predicted_class == 0:
                        current_stage = "down"
                    elif current_stage == "middle" and predicted_class == 1:
                        current_stage = "up"
                else:
                    current_stage = "middle"

                # print(f"Predicted class: {predicted_class}, Probability: {prediction_probability}, Current stage: {current_stage}")
            
                # Khoảng cách giữa 2 đầu gối 
                left_knee  = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]

                knee_width = calculate_distance(left_knee, right_knee)

                # Khoảng cách giữa 2 chân
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

                feet_width = calculate_distance(left_ankle, right_ankle)

                ratio = knee_width/feet_width
                # print(f"Knee width: {knee_width}, Feet width: {feet_width}, Ratio: {ratio}")
                # print("\n ------------------- \n")

                # Kiểm tra lỗi
                row = []
                isError = False
                if(current_stage == "down" and ratio < Min_down_ratio_knee_feet):
                    isError = True
                elif(current_stage == "middle" and ratio < Min_middle_ratio_knee_feet):
                    isError = True


                if isError:
                    row.append(3)
                    for lm in Important_kp:
                        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
                        row.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])

                if len(row) == len(df.columns):
                    if type == "train":
                        df.loc[len(df)] = row
                    elif type == "test":
                        df_test.loc[len(df_test)] = row
                    elif type == "validate":
                        df_va.loc[len(df_va)] = row
                else:
                    print(f"Lỗi! row có {len(row)} phần tử nhưng df có {len(df.columns)} cột.")


                # Nền
                cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

                # Display feet distance
                cv2.putText(image, "FEET", (15, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(feet_width, 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display knee distance
                cv2.putText(image, "KNEE", (95, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(knee_width, 2)), (90, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display label
                cv2.putText(image, "LABEL", (200, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(row[0], 2)), (200, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display current stage
                cv2.putText(image, "STAGE", (300, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, current_stage, (300, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)


            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            images.append(image)

            # Khi đủ 10 ảnh, hiển thị tất cả cùng lúc
            # if len(images) == max_images:
            #     fig, axes = plt.subplots(5, 2, figsize=(32, 18))  # Tạo grid 2x5
            #     for i, ax in enumerate(axes.flat):
            #         ax.imshow(images[i])
            #         ax.set_aspect('auto')
            #         ax.axis("off")  # Ẩn trục tọa độ
            #     plt.show()
            #     images = []  # Reset danh sách sau khi hiển thị


        Cap.release()

# detection_3_error()
# df

In [53]:
import os

# df = pd.DataFrame(columns=header)
# df_test = pd.DataFrame(columns=header)
# df_va = pd.DataFrame(columns=header)

#Path
video_folder = "Data/Train/Dau_goi_hep"
# video_folder = "Data/train/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_3_error(video_folder, video_file, "train")

video_folder = "Data/Test/Dau_goi_hep"
# video_folder = "Data/test/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_3_error(video_folder, video_file, "test")

video_folder = "Data/Validation/Dau_goi_hep"
# video_folder = "Data/validate/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_3_error(video_folder, video_file, "validate")

Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: li

In [54]:
df 

,label,NOSE_x,NOSE_y,NOSE_z,NOSE_visibility,LEFT_SHOULDER_x,LEFT_SHOULDER_y,LEFT_SHOULDER_z,LEFT_SHOULDER_visibility,RIGHT_SHOULDER_x,...,RIGHT_KNEE_z,RIGHT_KNEE_visibility,LEFT_ANKLE_x,LEFT_ANKLE_y,LEFT_ANKLE_z,LEFT_ANKLE_visibility,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_ANKLE_z,RIGHT_ANKLE_visibility
0,1.0,0.535062,0.249332,-0.298349,0.999890,0.567664,0.323744,-0.071113,0.999684,0.477610,...,-0.019138,0.992065,0.558640,0.903493,0.080078,0.970077,0.513414,0.908219,0.124308,0.983346
1,1.0,0.535019,0.256015,-0.305654,0.999889,0.567926,0.326266,-0.081536,0.999688,0.478063,...,-0.040403,0.992278,0.558611,0.903518,0.091800,0.970177,0.512646,0.909850,0.102087,0.983742
2,1.0,0.535222,0.263654,-0.307573,0.999891,0.567934,0.334535,-0.082262,0.999697,0.478092,...,-0.034905,0.992214,0.558633,0.901283,0.092424,0.970170,0.512343,0.909795,0.107190,0.983548
3,1.0,0.535212,0.279500,-0.308483,0.999881,0.567779,0.343994,-0.084086,0.999694,0.478039,...,-0.032856,0.991776,0.558793,0.898668,0.109657,0.968650,0.512348,0.909864,0.111147,0.982737
4,1.0,0.535222,0.287921,-0.310271,0.999887,0.567733,0.356836,-0.086673,0.999710,0.477941,...,-0.026602,0.991716,0.558939,0.897077,0.102962,0.969260,0.512467,0.910724,0.133688,0.982723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4826,3.0,0.504330,0.260842,-0.382953,0.999909,0.543674,0.351284,-0.163173,0.999913,0.445924,...,0.058406,0.990110,0.525992,0.901922,0.275802,0.963208,0.449529,0.912046,0.254963,0.984153
4827,3.0,0.504547,0.250975,-0.379222,0.999915,0.543685,0.343495,-0.156640,0.999911,0.445928,...,0.048499,0.990922,0.526442,0.901917,0.262613,0.966289,0.449512,0.911627,0.239959,0.985471
4828,3.0,0.503795,0.243799,-0.377042,0.999921,0.543679,0.335897,-0.149341,0.999910,0.445928,...,0.043275,0.991670,0.526730,0.902057,0.251831,0.969076,0.449465,0.911070,0.232682,0.986672
4829,3.0,0.503360,0.235697,-0.376520,0.999926,0.543650,0.329100,-0.140820,0.999910,0.445916,...,0.036877,0.992338,0.527000,0.902684,0.247598,0.971522,0.449278,0.911076,0.225677,0.987740


# 4.Lỗi xuống quá xâu

In [55]:
#Xử lý từng video để tính khoảng cách chân, đàu gốin 
def detection_4_error(Video_folder, Video_name, type):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích frame như ảnh đầu vào
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            # Chuyển lại ảnh sang BGR để hiển thị
            # image.flags.writeable = True
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            #khởi tạo biến tính
            knee_angle = None
            hip_angle = None

            
            try:
                # Lấy kp từ frame
                row = extract_important_keypoints(results)
                X = pd.DataFrame([row], columns=header[1:])

                # Tinhs toán và so sánh góc giữa 2 đầu gối
                landmarks = results.pose_landmarks.landmark

                # Góc đầu gối trái 
                left_knee  = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z]
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z]
                left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)

                # Góc đầu gối phải
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z]

                right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)

                #Láy trung bình cộng 2 góc
                knee_angle = (left_knee_angle + right_knee_angle) / 2

                row = []
                if(knee_angle < Min_knee_angle):
                    row.append(4)
                    for lm in Important_kp:
                        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
                        row.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
                
                if len(row) == len(df.columns):
                    if type == "train":
                        df.loc[len(df)] = row
                    elif type == "test":
                        df_test.loc[len(df_test)] = row
                    elif type == "validate":
                        df_va.loc[len(df_va)] = row
                else:
                    print(f"Lỗi! row có {len(row)} phần tử nhưng df có {len(df.columns)} cột.")

                # Nền
                cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

                #Display label
                cv2.putText(image, "LABEL", (10, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(row[0], 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display knee angle 
                cv2.putText(image, "knee_angle", (150, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(knee_angle, 2)), (150, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            # if(knee_angle < 55):
            #     plt.imshow(image)
            #     plt.axis("off")  # Ẩn trục tọa độ
            #     plt.show()

        Cap.release()

# Đường dẫn thư mục chứa video
# video_folder = "Data/Train/Correct"
# video_files = "20250228_150821.mp4"

# video_folder = "Data/Train/Gap_lung"
# video_files = "20250228_165326.mp4"

# video_folder = "Data/Train/Xuong_qua_xau"
# video_files = "20250228_163416.mp4"

# process_frame(video_folder, video_files)



In [56]:
import os

# df = pd.DataFrame(columns=header)
# df_test = pd.DataFrame(columns=header)
# df_va = pd.DataFrame(columns=header)

#Path
video_folder = "Data/Train/Xuong_qua_xau"
# video_folder = "Data/train/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_4_error(video_folder, video_file, "train")

video_folder = "Data/Test/Xuong_qua_xau"
# video_folder = "Data/test/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_4_error(video_folder, video_file, "test")

video_folder = "Data/Validation/Xuong_qua_xau"
# video_folder = "Data/validate/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_4_error(video_folder, video_file, "validate")

Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: li

In [57]:
df

,label,NOSE_x,NOSE_y,NOSE_z,NOSE_visibility,LEFT_SHOULDER_x,LEFT_SHOULDER_y,LEFT_SHOULDER_z,LEFT_SHOULDER_visibility,RIGHT_SHOULDER_x,...,RIGHT_KNEE_z,RIGHT_KNEE_visibility,LEFT_ANKLE_x,LEFT_ANKLE_y,LEFT_ANKLE_z,LEFT_ANKLE_visibility,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_ANKLE_z,RIGHT_ANKLE_visibility
0,1.0,0.535062,0.249332,-0.298349,0.999890,0.567664,0.323744,-0.071113,0.999684,0.477610,...,-0.019138,0.992065,0.558640,0.903493,0.080078,0.970077,0.513414,0.908219,0.124308,0.983346
1,1.0,0.535019,0.256015,-0.305654,0.999889,0.567926,0.326266,-0.081536,0.999688,0.478063,...,-0.040403,0.992278,0.558611,0.903518,0.091800,0.970177,0.512646,0.909850,0.102087,0.983742
2,1.0,0.535222,0.263654,-0.307573,0.999891,0.567934,0.334535,-0.082262,0.999697,0.478092,...,-0.034905,0.992214,0.558633,0.901283,0.092424,0.970170,0.512343,0.909795,0.107190,0.983548
3,1.0,0.535212,0.279500,-0.308483,0.999881,0.567779,0.343994,-0.084086,0.999694,0.478039,...,-0.032856,0.991776,0.558793,0.898668,0.109657,0.968650,0.512348,0.909864,0.111147,0.982737
4,1.0,0.535222,0.287921,-0.310271,0.999887,0.567733,0.356836,-0.086673,0.999710,0.477941,...,-0.026602,0.991716,0.558939,0.897077,0.102962,0.969260,0.512467,0.910724,0.133688,0.982723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5444,4.0,0.520781,0.521900,-0.329564,0.999978,0.559292,0.587115,-0.161914,0.999950,0.462434,...,-0.250277,0.985218,0.542599,0.873931,-0.048878,0.952986,0.463122,0.874718,-0.060643,0.983794
5445,4.0,0.520701,0.519965,-0.328821,0.999978,0.559293,0.585739,-0.161511,0.999950,0.462416,...,-0.252293,0.985282,0.542565,0.873895,-0.048871,0.953160,0.463121,0.874642,-0.060532,0.983834
5446,4.0,0.520794,0.517375,-0.329103,0.999979,0.559291,0.584356,-0.161809,0.999950,0.462357,...,-0.258186,0.985379,0.542477,0.873903,-0.051388,0.953574,0.463121,0.874641,-0.061068,0.983870
5447,4.0,0.520793,0.509946,-0.327740,0.999979,0.559272,0.580355,-0.159747,0.999949,0.462337,...,-0.259302,0.985617,0.542383,0.873931,-0.054825,0.954019,0.463132,0.874642,-0.061632,0.984041


# 5. Lỗi gập lưng


In [58]:
def detection_5_error(Video_folder, Video_name, type):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích frame như ảnh đầu vào
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            # Chuyển lại ảnh sang BGR để hiển thị
            # image.flags.writeable = True
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            #khởi tạo biến tính
            knee_angle = None
            hip_angle = None

            
            try:
                # Lấy kp từ frame
                row = extract_important_keypoints(results)
                X = pd.DataFrame([row], columns=header[1:])

                # Tinhs toán và so sánh góc giữa 2 đầu gối
                landmarks = results.pose_landmarks.landmark

                # Góc đầu gối trái 
                left_knee  = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z]
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z]
                left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)

                # Góc đầu gối phải
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z]

                right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)

                #Láy trung bình cộng 2 góc
                knee_angle = (left_knee_angle + right_knee_angle) / 2

                #Góc hông phải
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z]

                right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)

                #Góc hông trái
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z]
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]
                left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z]

                left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)

                #Lấy trung bình cộng 2 góc
                hip_angle = (left_hip_angle + right_hip_angle) / 2

                ratio = knee_angle/hip_angle

                #Kiểm tra lỗi
                row = []
                if(ratio > Max_ratio_knee_hip):
                    row.append(5)
                    for lm in Important_kp:
                        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
                        row.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
                
                if len(row) == len(df.columns):
                    if type == "train":
                        df.loc[len(df)] = row
                    elif type == "test":
                        df_test.loc[len(df_test)] = row
                    elif type == "validate":
                        df_va.loc[len(df_va)] = row
                else:
                    print(f"Lỗi! row có {len(row)} phần tử nhưng df có {len(df.columns)} cột.")

                # Nền
                cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

                # Display hip angle
                cv2.putText(image, "hip_angle", (15, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(hip_angle, 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display knee angle 
                cv2.putText(image, "knee_ankle", (150, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(knee_angle, 2)), (150, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            # Hiển thị ảnh
            # if(hip_angle/shoulder_hip_ankle_angle <0.52):
            #     plt.imshow(image)
            #     plt.axis("off")  # Ẩn trục tọa độ
            #     plt.show()

            # plt.imshow(image)
            # plt.axis("off")  # Ẩn trục tọa độ
            # plt.show()

        Cap.release()

In [59]:
import os

# df = pd.DataFrame(columns=header)
# df_test = pd.DataFrame(columns=header)
# df_va = pd.DataFrame(columns=header)

#Path
# video_folder = "Data/train/Xuong_qua_xau"
video_folder = "Data/Train/Gap_lung"
# video_folder = "Data/train/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_5_error(video_folder, video_file, "train")

video_folder = "Data/Test/Gap_lung"
# video_folder = "Data/test/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_5_error(video_folder, video_file, "test")

video_folder = "Data/Validation/Gap_lung"
# video_folder = "Data/validate/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_5_error(video_folder, video_file, "validate")

Lỗi! row có 0 phần tử nhưng df có 37 cột.
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Lỗi! row có 0 phần tử nhưng df có 

In [60]:
df

,label,NOSE_x,NOSE_y,NOSE_z,NOSE_visibility,LEFT_SHOULDER_x,LEFT_SHOULDER_y,LEFT_SHOULDER_z,LEFT_SHOULDER_visibility,RIGHT_SHOULDER_x,...,RIGHT_KNEE_z,RIGHT_KNEE_visibility,LEFT_ANKLE_x,LEFT_ANKLE_y,LEFT_ANKLE_z,LEFT_ANKLE_visibility,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_ANKLE_z,RIGHT_ANKLE_visibility
0,1.0,0.535062,0.249332,-0.298349,0.999890,0.567664,0.323744,-0.071113,0.999684,0.477610,...,-0.019138,0.992065,0.558640,0.903493,0.080078,0.970077,0.513414,0.908219,0.124308,0.983346
1,1.0,0.535019,0.256015,-0.305654,0.999889,0.567926,0.326266,-0.081536,0.999688,0.478063,...,-0.040403,0.992278,0.558611,0.903518,0.091800,0.970177,0.512646,0.909850,0.102087,0.983742
2,1.0,0.535222,0.263654,-0.307573,0.999891,0.567934,0.334535,-0.082262,0.999697,0.478092,...,-0.034905,0.992214,0.558633,0.901283,0.092424,0.970170,0.512343,0.909795,0.107190,0.983548
3,1.0,0.535212,0.279500,-0.308483,0.999881,0.567779,0.343994,-0.084086,0.999694,0.478039,...,-0.032856,0.991776,0.558793,0.898668,0.109657,0.968650,0.512348,0.909864,0.111147,0.982737
4,1.0,0.535222,0.287921,-0.310271,0.999887,0.567733,0.356836,-0.086673,0.999710,0.477941,...,-0.026602,0.991716,0.558939,0.897077,0.102962,0.969260,0.512467,0.910724,0.133688,0.982723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6458,5.0,0.502140,0.413816,-0.473057,0.999984,0.546305,0.430141,-0.230609,0.999992,0.452828,...,-0.158037,0.991018,0.543855,0.900754,-0.048971,0.973336,0.457092,0.896660,-0.022327,0.990995
6459,5.0,0.502404,0.397786,-0.456841,0.999981,0.546309,0.419557,-0.214265,0.999989,0.453119,...,-0.166889,0.991491,0.543796,0.900634,-0.033763,0.974921,0.457090,0.896852,-0.017943,0.991395
6460,5.0,0.503870,0.380777,-0.428196,0.999976,0.546271,0.409370,-0.190015,0.999986,0.453118,...,-0.151754,0.991962,0.543606,0.900461,-0.033545,0.976584,0.457072,0.896918,-0.013037,0.991832
6461,5.0,0.503971,0.366431,-0.401071,0.999970,0.546266,0.398942,-0.168638,0.999982,0.453083,...,-0.130439,0.992415,0.543525,0.900597,-0.027711,0.978049,0.457071,0.897068,-0.004526,0.992231


# 6. Thêm data đúng

In [61]:
def detection_correct(Video_folder, Video_name, type):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích frame như ảnh đầu vào
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            # Chuyển lại ảnh sang BGR để hiển thị
            # image.flags.writeable = True
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            #khởi tạo biến tính
            knee_angle = None
            hip_angle = None

            
            try:
                # Lấy kp từ frame
                landmarks = results.pose_landmarks.landmark

                #Kiểm tra lỗi
                row = []

                row.append(0)
                for lm in Important_kp:
                    keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
                    row.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
                
                if len(row) == len(df.columns):
                    if type == "train":
                        df.loc[len(df)] = row
                    elif type == "test":
                        df_test.loc[len(df_test)] = row
                    elif type == "validate":
                        df_va.loc[len(df_va)] = row
                else:
                    print(f"Lỗi! row có {len(row)} phần tử nhưng df có {len(df.columns)} cột.")


            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            # Hiển thị ảnh
            # if(hip_angle/shoulder_hip_ankle_angle <0.52):
            #     plt.imshow(image)
            #     plt.axis("off")  # Ẩn trục tọa độ
            #     plt.show()

            # plt.imshow(image)
            # plt.axis("off")  # Ẩn trục tọa độ
            # plt.show()

        Cap.release()


In [62]:
import os

#Path
video_folder = "Data/Train/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_correct(video_folder, video_file, "train")

video_folder = "Data/Test/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_correct(video_folder, video_file, "test")

video_folder = "Data/Validation/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_correct(video_folder, video_file, "validate")


# 7. Đưa dataframe ra csv

In [63]:
def save_splits_to_csv(train_df, test_df, val_df, output_dir="dataset"):
    """
    Lưu các DataFrame train, test, validation ra file CSV.

    Args:
        train_df (pd.DataFrame): Dữ liệu train
        test_df (pd.DataFrame): Dữ liệu test
        val_df (pd.DataFrame): Dữ liệu validation
        output_dir (str): Thư mục lưu file CSV (mặc định là 'output')
    """
    import os
    os.makedirs(output_dir, exist_ok=True)  # Tạo thư mục nếu chưa có
    
    train_df.to_csv(f"{output_dir}/train.csv", index=False)
    test_df.to_csv(f"{output_dir}/test.csv", index=False)
    val_df.to_csv(f"{output_dir}/validation.csv", index=False)
    
    print(f"Dữ liệu đã được lưu vào thư mục '{output_dir}'.")

In [64]:
save_splits_to_csv(df, df_test, df_va, "dataset")

Dữ liệu đã được lưu vào thư mục 'dataset'.
